# 俄乌战争虚假信息与谣言传播分析 - 数据准备与抽取

## 1. 导入必要的库

In [3]:
import os
import pandas as pd
import glob
import gzip
from pathlib import Path
import gc

2. 定义数据路径与事件节点

In [6]:
# 定义数据目录
data_dir = Path('./timeline')  # 修改为实际路径
output_dir = Path('./timeline/combined')   # 修改为实际路径

# 获取所有事件子目录
event_dirs = [d for d in data_dir.iterdir() if d.is_dir()]

print(f"发现 {len(event_dirs)} 个事件目录。")
for event in event_dirs:
    print(event.name)

# 创建输出目录（如果不存在）
output_dir.mkdir(parents=True, exist_ok=True)

发现 8 个事件目录。
20220227-0302
20230518-0524
20220518-0524
20220930-1006
20221109-1115
20220623-0701
20220330-0405
20230301-0305


In [7]:
# 定义需要读取的列
columns_to_use = [
    "userid", "username", "acctdesc", "location", "following", "followers", "totaltweets",
    "usercreatedts", "tweetid", "tweetcreatedts", "retweetcount", "text", "hashtags",
    "language", "coordinates", "favorite_count", "is_retweet",
    "original_tweet_id", "original_tweet_userid", "original_tweet_username",
    "in_reply_to_status_id", "in_reply_to_user_id", "in_reply_to_screen_name",
    "is_quote_status", "quoted_status_id", "quoted_status_userid", "quoted_status_username",
    "extractedts"
]

for event_dir in event_dirs:
    print(f"\n处理事件目录: {event_dir.name}")
    
    # 获取该事件目录下所有CSV文件（支持gzip压缩的CSV）
    csv_files = list(event_dir.glob('*.csv')) + list(event_dir.glob('*.csv.gz'))
    
    if not csv_files:
        print(f"  - 没有找到CSV文件在 {event_dir.name}")
        continue
    
    dataframe_collection = []
    
    for csv_file in csv_files:
        try:
            if csv_file.suffix == '.gz':
                # 读取gzip压缩的CSV文件
                df = pd.read_csv(
                    csv_file,
                    compression='gzip',
                    encoding='utf-8',
                    usecols=columns_to_use,
                    engine='python',
                    quoting=1  # csv.QUOTE_ALL
                )
            else:
                # 读取普通的CSV文件
                df = pd.read_csv(
                    csv_file,
                    encoding='utf-8',
                    
                    engine='python',
                    quoting=1  # csv.QUOTE_ALL
                )
            
            dataframe_collection.append(df)
            print(f"  - 成功读取文件: {csv_file.name}，包含 {len(df)} 条推文。")
        except Exception as e:
            print(f"  - 读取文件 {csv_file.name} 时出错: {e}")
    
    if not dataframe_collection:
        print(f"  - 没有成功读取任何文件在 {event_dir.name}")
        continue
    
    # 合并所有DataFrame
    try:
        df_combined = pd.concat(dataframe_collection, axis=0, ignore_index=True)
        print(f"  - 合并后的DataFrame包含 {len(df_combined)} 条推文。")
    except Exception as e:
        print(f"  - 合并DataFrame时出错: {e}")
        continue
    
    # 转换时间戳字段
    df_combined['tweetcreatedts'] = pd.to_datetime(df_combined['tweetcreatedts'], errors='coerce')
    df_combined['extractedts'] = pd.to_datetime(df_combined['extractedts'], errors='coerce')
    
    
        # 按'tweetcreatedts'排序
    df_combined.sort_values(by='tweetcreatedts', inplace=True)
    
    # 重置索引
    df_combined.reset_index(inplace=True, drop=True)
    
    # 定义输出文件路径
    output_file = output_dir / f"{event_dir.name}_merged.csv.gz"  # 保存为压缩CSV
    # 如果需要保存为未压缩的CSV，可以使用以下行并注释上面一行
    # output_file = output_dir / f"{event_dir.name}_merged.csv"
    
    try:
        df_combined.to_csv(output_file, index=False, compression='gzip')  # 使用gzip压缩
        # df_combined.to_csv(output_file, index=False)  # 未压缩保存
        print(f"  - 成功保存合并并排序的数据到: {output_file}")
    except Exception as e:
        print(f"  - 保存合并数据到 {output_file} 时出错: {e}")
    
    # 清理内存
    del df_combined
    gc.collect()

print("\n所有事件的数据处理完成。")


处理事件目录: 20220227-0302
  - 成功读取文件: UkraineCombinedTweetsDeduped_FEB28_part2.csv，包含 140384 条推文。
  - 成功读取文件: UkraineCombinedTweetsDeduped_FEB28_part1.csv，包含 237787 条推文。
  - 成功读取文件: UkraineCombinedTweetsDeduped_FEB27.csv，包含 357018 条推文。
  - 成功读取文件: UkraineCombinedTweetsDeduped_MAR01.csv，包含 409279 条推文。
  - 成功读取文件: UkraineCombinedTweetsDeduped_MAR02.csv，包含 417031 条推文。
  - 合并后的DataFrame包含 1561499 条推文。
  - 成功保存合并并排序的数据到: timeline/combined/20220227-0302_merged.csv.gz

处理事件目录: 20230518-0524
  - 成功读取文件: 20230518_to_20230520_UkraineCombinedTweetsDeduped.csv，包含 132618 条推文。
  - 成功读取文件: 20230523_UkraineCombinedTweetsDeduped.csv，包含 26077 条推文。
  - 成功读取文件: 20230524_UkraineCombinedTweetsDeduped.csv，包含 21954 条推文。
  - 成功读取文件: 20230522_UkraineCombinedTweetsDeduped.csv，包含 39176 条推文。
  - 成功读取文件: 20230521_UkraineCombinedTweetsDeduped.csv，包含 52147 条推文。
  - 合并后的DataFrame包含 271972 条推文。
  - 成功保存合并并排序的数据到: timeline/combined/20230518-0524_merged.csv.gz

处理事件目录: 20220518-0524
  - 成功读取文件: 0519_UkraineCombinedTweetsDed